In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/AhmedSalahBasha/SchemaMatching.git

Cloning into 'SchemaMatching'...
remote: Enumerating objects: 44, done.
remote: Total 44 (delta 0), reused 0 (delta 0), pack-reused 44
Unpacking objects: 100% (44/44), done.


In [ ]:
!git clone https://github.com/davidfoerster/schema-matching.git

Cloning into 'schema-matching'...
remote: Enumerating objects: 1498, done.
remote: Total 1498 (delta 0), reused 0 (delta 0), pack-reused 1498
Receiving objects: 100% (1498/1498), 266.23 KiB | 1.71 MiB/s, done.
Resolving deltas: 100% (964/964), done.


In [ ]:
!./content/drive/MyDrive/Colab Notebooks/schema-matching/demo/1-match.sh

bash: /content/drive/MyDrive/Colab: No such file or directory


In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 983 kB 8.4 MB/s 
     |████████████████████████████████| 358 kB 54.2 MB/s 
     |████████████████████████████████| 138 kB 57.3 MB/s 
     |████████████████████████████████| 4.0 MB 43.4 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 58 kB 5.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
from lxml import html
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

ModuleNotFoundError: ignored

In [ ]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

#driver.get("https://www.webite-url.com")

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
class HtmlGetter:
   def get_html(self, url):
       pass

class HtmlParseGetter(HtmlGetter):
   def __init__(self, subject):
       self.subject = subject
   def get_html(self, url):
       html_source = self.subject.get_html(url)
       html_element = html.fromstring(html_source)
       return html_element

class SeleniumHtmlGetter(HtmlGetter):
   def __init__(self, scroll_to_bottom=False):
       self.scroll_to_bottom = scroll_to_bottom
   def get_html(self, url):
      chrome_options = webdriver.ChromeOptions()
      chrome_options.add_argument('--headless')
      chrome_options.add_argument('--no-sandbox')
      chrome_options.add_argument('--disable-dev-shm-usage')
      browser = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
      browser.maximize_window()
      browser.get(url)
      html_source = browser.page_source
      browser.quit()
      return html_source

if __name__ == '__main__':
   url = 'https://tiki.vn/dien-thoai-smartphone/c1795?page=2'
   html_getter = HtmlParseGetter(SeleniumHtmlGetter( ))
   html_tree = html_getter.get_html(url)
   for v in html_tree.xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "product-item", " " ))]'):
     print(v.xpath('//a/@href'))
     break


In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import json
from bs4 import BeautifulSoup

In [ ]:
import sys
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

with open("tiki_crawl.txt", mode='w', encoding='utf8') as f:
  for index in range(1,10):
    response = requests.get(f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=48&include=advertisement&aggregations=2&trackity_id=d743e974-1a06-9329-8e89-e909bdbed2ae&category=1795&page={index}&urlKey=dien-thoai-smartphone", headers = headers)
    datas_json = json.loads(response.text)['data']
    for idx in range(len(datas_json)):
      productId = datas_json[idx]['id']
      response_detail = requests.get(f"https://tiki.vn/api/v2/products/{productId}", headers = headers)

      phone_detail = dict()
      phone_detail["Tên sản phẩm"] = response_detail.json()['name']
      phone_detail["Giá"] = response_detail.json()['price']
      for features in response_detail.json()['specifications']:
        for feature in features['attributes']:
          value = BeautifulSoup(feature['value'], "lxml").text.strip()
          phone_detail[feature['name']] = value
      f.write(json.dumps(phone_detail, ensure_ascii=False) + "\n")

In [70]:
columns = set() 
with open("/content/tiki_crawl.txt", mode = 'r', encoding = 'utf8') as g:
  for idx, line in enumerate(g):
    line = json.loads(line)
    columns.update(set(line.keys()))

In [74]:
import pandas as pd
df = pd.DataFrame(columns = columns)

In [75]:
df

,Kết nối khác,Cổng sạc,Camera sau,Độ phân giải,Chất liệu,Phụ kiện đi kèm,Băng tần 3G,Model,Chức năng khác,Thương hiệu,...,Hỗ trợ 4G,Lưu ý,Nghe nhạc,Giá,Pin có thể tháo rời,Hỗ trợ 5G,Hỗ trợ thẻ nhớ ngoài,Chip set,Bluetooth,Ghi âm


In [77]:
with open("/content/tiki_crawl.txt", mode = 'r', encoding = 'utf8') as g:
  for idx, line in enumerate(g):
    line = json.loads(line)
    df = df.append(line, ignore_index=True)

In [78]:
df.head()

,Kết nối khác,Cổng sạc,Camera sau,Độ phân giải,Chất liệu,Phụ kiện đi kèm,Băng tần 3G,Model,Chức năng khác,Thương hiệu,...,Hỗ trợ 4G,Lưu ý,Nghe nhạc,Giá,Pin có thể tháo rời,Hỗ trợ 5G,Hỗ trợ thẻ nhớ ngoài,Chip set,Bluetooth,Ghi âm
0,NaN,Micro USB,13 MP,HD+ (700 x 1600 pixels),NaN,"Cáp, sạc, sách hướng dẫn",NaN,Redmi 9A,NaN,Xiaomi,...,Có,NaN,Có,2190000,Pin liền,NaN,MicroSD,NaN,"A2DP, LE",NaN
1,NaN,USB Type-C,48.0 MP + 5.0 MP + 2.0 MP + 2.0 MP,HD+ (720 x 1600 pixels),NaN,"Sách hướng dẫn, cây lấy sim, cáp",NaN,M12,NaN,Samsung,...,Có,NaN,Có,3890000,NaN,NaN,MicroSD,Exynos 850 (8 nhân),Bluetooth v5.0,NaN
2,NaN,Type-C,NaN,Full HD+ (1080 x 2400 Pixels),Khung & Mặt lưng nhựa,NaN,NaN,M32,NaN,Samsung,...,NaN,NaN,NaN,4660000,NaN,NaN,NaN,MediaTek Helio G80 8 nhân,A2DP LE v5.0,NaN
3,NaN,Type - C,Bộ 3 Camera sau AI 48MP.,Full HD+ (1080 x 2400 Pixels),NaN,Thiết bị | Bộ sạc | Cáp USB Type-C | Dụng cụ t...,NaN,NaN,Sạc pin nhanh,Xiaomi,...,NaN,NaN,NaN,5990000,NaN,NaN,có,MediaTek Dimensity 700,Bluetooth 5.1,Có
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Samsung,...,NaN,NaN,NaN,4890000,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Kết nối khác              97 non-null     object
 1   Cổng sạc                  313 non-null    object
 2   Camera sau                101 non-null    object
 3   Độ phân giải              246 non-null    object
 4   Chất liệu                 201 non-null    object
 5   Phụ kiện đi kèm           246 non-null    object
 6   Băng tần 3G               10 non-null     object
 7   Model                     282 non-null    object
 8   Chức năng khác            75 non-null     object
 9   Thương hiệu               402 non-null    object
 10  Jack tai nghe             297 non-null    object
 11  Xuất xứ thương hiệu       132 non-null    object
 12  Có thuế VAT               325 non-null    object
 13  Trọng lượng               297 non-null    object
 14  Kích thước màn hình       

In [81]:
df.isnull().sum()

Kết nối khác                305
Cổng sạc                     89
Camera sau                  301
Độ phân giải                156
Chất liệu                   201
Phụ kiện đi kèm             156
Băng tần 3G                 392
Model                       120
Chức năng khác              327
Thương hiệu                   0
Jack tai nghe               105
Xuất xứ thương hiệu         270
Có thuế VAT                  77
Trọng lượng                 105
Kích thước màn hình         108
Loại Sim                     91
FM radio                    330
Băng tần 2G                 395
Loại/ Công nghệ màn hình    116
Tên sản phẩm                  0
Hỗ trợ thẻ tối đa           207
Xuất xứ                     276
Thời gian sạc               395
Tốc độ CPU                  144
Camera trước                301
Mã part                     401
Chip đồ họa (GPU)           270
ROM                         215
Đèn Flash                   342
GPS                         320
Quay phim                   316
NFC     

In [82]:
df.to_csv("tiki_crawl.csv", sep='\t', encoding='utf-8')

In [87]:
fd = pd.read_csv("/content/tiki_crawl.csv", sep='\t', encoding = 'utf8', index_col=0)

In [88]:
fd

,Kết nối khác,Cổng sạc,Camera sau,Độ phân giải,Chất liệu,Phụ kiện đi kèm,Băng tần 3G,Model,Chức năng khác,Thương hiệu,...,Hỗ trợ 4G,Lưu ý,Nghe nhạc,Giá,Pin có thể tháo rời,Hỗ trợ 5G,Hỗ trợ thẻ nhớ ngoài,Chip set,Bluetooth,Ghi âm
0,NaN,Micro USB,13 MP,HD+ (700 x 1600 pixels),NaN,"Cáp, sạc, sách hướng dẫn",NaN,Redmi 9A,NaN,Xiaomi,...,Có,NaN,Có,2190000,Pin liền,NaN,MicroSD,NaN,"A2DP, LE",NaN
1,NaN,USB Type-C,48.0 MP + 5.0 MP + 2.0 MP + 2.0 MP,HD+ (720 x 1600 pixels),NaN,"Sách hướng dẫn, cây lấy sim, cáp",NaN,M12,NaN,Samsung,...,Có,NaN,Có,3890000,NaN,NaN,MicroSD,Exynos 850 (8 nhân),Bluetooth v5.0,NaN
2,NaN,Type-C,NaN,Full HD+ (1080 x 2400 Pixels),Khung & Mặt lưng nhựa,NaN,NaN,M32,NaN,Samsung,...,NaN,NaN,NaN,4660000,NaN,NaN,NaN,MediaTek Helio G80 8 nhân,A2DP LE v5.0,NaN
3,NaN,Type - C,Bộ 3 Camera sau AI 48MP.,Full HD+ (1080 x 2400 Pixels),NaN,Thiết bị | Bộ sạc | Cáp USB Type-C | Dụng cụ t...,NaN,NaN,Sạc pin nhanh,Xiaomi,...,NaN,NaN,NaN,5990000,NaN,NaN,có,MediaTek Dimensity 700,Bluetooth 5.1,Có
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Samsung,...,NaN,NaN,NaN,4890000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,NFC,Lightning,3 camera 12 MP,1284 x 2778 pixels,Khung thép không gỉ & Mặt lưng kính cường lực,NaN,NaN,iPhone 13 Pro Max 128GB,NaN,Apple,...,Có,NaN,Có,29490000,Pin liền,NaN,NaN,Chip A15 Bionic (5nm),Có,NaN
398,NaN,Lightning,12MP,750 x 1334 pixels,NaN,"Sách hướng dẫn, cáp",NaN,NaN,NaN,Apple,...,Có,NaN,Có,11750000,Pin liền,Có,NaN,Apple A15 Bionic,Có,NaN
399,NaN,Lightning,Dual Camera 12.0 MP,HD (828 x 1792 pixels),NaN,Cáp sạc + Sách HDSD,Có,NaN,"Hỗ trợ sạc nhanh, chống nước chuẩn IP67, Face ID",Apple,...,Có,NaN,Có,11750000,Pin liền,NaN,NaN,Apple A13 Bionic,v5.0,Có
400,NaN,Lightning,Dual Camera 12.0 MP,HD (828 x 1792 pixels),NaN,Cáp sạc + Sách HDSD,Có,NaN,"Hỗ trợ sạc nhanh, chống nước chuẩn IP67, Face ID",Apple,...,Có,NaN,Có,11750000,Pin liền,NaN,NaN,Apple A13 Bionic,v5.0,Có
